# Cat or Dog?

Using neural networks to determine if an image contains a cat or a dog.

## Project Goal

This project aims to make a image classification model tha can determine if an image contains a cat or a dog.

## Definition(s)

- CNN/ convolutional neural net: A model that tries to observe patterns in an array/image and then use those patterns to identify classes.

## Data

This dataset was taken from an old kaggle competition that can be found [here](https://www.kaggle.com/c/dogs-vs-cats). The images are from an old CAPTCHA known as *Animal Species Image Recognition for Restricting Access* (or *Asirra* for short) which asked the user to identify whether or not several images displayed contained either a cat or a dog.

There are 25,000 images total evenly split between cats(0) and dogs(1). 

The train and validation data were organized and divided up as such:

In [ ]:
filenames = os.listdir(data_path)
labels = \[\]

for filename in filenames:
    label = filename.split('.')\[0\]
    if label == 'dog':
        labels.append(1) #1 for dog
    else:
        labels.append(0) #0 for cat

df = pd.DataFrame({
    'filename': filenames,
    'label': labels
})

X_t, v_t = train_test_split(df, test_size = 0.15,  random_state = 7)
X_t = X_t.reset_index(drop = True)
v_t = v_t.reset_index(drop=True)

#X_t is the training data and v_t is the validation data

## EDA

In order to get a grasp on the rough size of the images to create a better target size for my models I had made this script to get the average height and width of the images:

In [ ]:
from PIL import Image

width = []
height = []

for filename in filenames:
    

    
    img_path = os.path.join(data_path, filename)
    im = Image.open(img_path)
    
    w, h = im.size
    
    width.append(w)
    height.append(h)
    
w_avg = sum(width)/len(width)
h_avg = sum(height)/len(height)
    
print(w_avg, h_avg)

This returned (404, 360)

## Modeling

All modeling was done using CNN's with convolutional and dense layers. Dropout layers were used to avoid overfitting. Accuracy score was the evaluation metric.

#### FSM

At this time I was not aware that I'd made an error with the average image size (which I was using as the target size). The first script only grabbed the dimensions of the last image in the diretcory, this error is addresed in the 'Modeling' notebook.

In [ ]:
img_gen = ImageDataGenerator(rescale = 1./255., horizontal_flip = True) 

train_generator = img_gen.flow_from_dataframe(
    X_t,
    data_path,
    x_col = 'filename',
    y_col = 'label',
    target_size = (499, 375), 
    batch_size = batch_size,
    class_mode ='categorical'
)

val_img_gen = ImageDataGenerator(rescale = 1./255.) 

val_generator = img_gen.flow_from_dataframe(
    v_t,
    data_path,
    x_col = 'filename',
    y_col = 'label',
    target_size = (499, 375), 
    batch_size = batch_size,
    class_mode ='categorical'
)

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

fsm = Sequential() 
fsm.add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (width, height, 3)))#3 for rgb
fsm.add(MaxPooling2D(pool_size=(2, 2)))
fsm.add(BatchNormalization())
fsm.add(Dropout(0.20))

fsm.add(Conv2D(64, (3, 3), activation = 'relu'))
fsm.add(MaxPooling2D((2, 2)))
fsm.add(BatchNormalization())
fsm.add(Dropout(0.20))

fsm.add(Conv2D(128, (3, 3), activation = 'relu'))
fsm.add(MaxPooling2D((2, 2)))
fsm.add(BatchNormalization())
fsm.add(Dropout(0.20))

fsm.add(Flatten())
fsm.add(Dense(64, activation='relu'))
fsm.add(BatchNormalization())
fsm.add(Dropout(0.20))

fsm.add(Dense(128, activation='relu'))
fsm.add(BatchNormalization())
fsm.add(Dropout(0.20))

fsm.add(Dense(2, activation='softmax')) #2 for cat or dog 
fsm.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# *IMAGE HERE*

#### Alpha

This model uses the actual average image height and width and all the values of the conv2d and dense layers were doubled to make them more able to extract features and 

In [ ]:
img_gen = ImageDataGenerator(rescale = 1./255., horizontal_flip = True)

train_generator = img_gen.flow_from_dataframe(
    X_t,
    data_path,
    x_col = 'filename',
    y_col = 'label',
    target_size = img_size,#
    batch_size = batch_size,
    class_mode ='categorical'
)

val_img_gen = ImageDataGenerator(rescale = 1./255.)

val_generator = img_gen.flow_from_dataframe(
    v_t,
    data_path,
    x_col = 'filename',
    y_col = 'label',
    target_size = img_size,
    batch_size = batch_size,
    class_mode ='categorical'
)

alpha = Sequential() 
alpha.add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (width, height, 3)))#3 for rgb
alpha.add(MaxPooling2D(pool_size=(2, 2)))
alpha.add(BatchNormalization())
alpha.add(Dropout(0.20))

alpha.add(Conv2D(64, (3, 3), activation = 'relu'))
alpha.add(MaxPooling2D((2, 2)))
alpha.add(BatchNormalization())
alpha.add(Dropout(0.20))

alpha.add(Conv2D(128, (3, 3), activation = 'relu'))
alpha.add(MaxPooling2D((2, 2)))
alpha.add(BatchNormalization())
alpha.add(Dropout(0.20))

alpha.add(Conv2D(256, (3, 3), activation = 'relu'))
alpha.add(MaxPooling2D((2, 2)))
alpha.add(BatchNormalization())
alpha.add(Dropout(0.20))

alpha.add(Flatten())
alpha.add(Dense(128, activation='relu'))
alpha.add(BatchNormalization())
alpha.add(Dropout(0.20))

alpha.add(Dense(256, activation='relu'))
alpha.add(BatchNormalization())
alpha.add(Dropout(0.20))

alpha.add(Dense(2, activation='softmax')) #2 for cat or dog
alpha.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

alpha.summary()

# *IMAGE HERE*

#### Delta

In [ ]:
img_size = (256, 256)

train_generator = img_gen.flow_from_dataframe(
    X_t,
    data_path,
    x_col = 'filename',
    y_col = 'label',
    target_size = img_size,
    batch_size = batch_size,
    class_mode ='categorical'
)

val_img_gen = ImageDataGenerator(rescale = 1./255.)

val_generator = img_gen.flow_from_dataframe(
    v_t,
    data_path,
    x_col = 'filename',
    y_col = 'label',
    target_size = img_size,
    batch_size = batch_size,
    class_mode ='categorical'
)

delta = Sequential() 
delta.add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (256, 256, 3)))#3 for rgb
delta.add(MaxPooling2D(pool_size=(2, 2)))
delta.add(BatchNormalization())
delta.add(Dropout(0.20))

delta.add(Conv2D(64, (3, 3), activation = 'relu'))
delta.add(MaxPooling2D((2, 2)))
delta.add(BatchNormalization())
delta.add(Dropout(0.20))

delta.add(Conv2D(128, (3, 3), activation = 'relu'))
delta.add(MaxPooling2D((2, 2)))
delta.add(BatchNormalization())
delta.add(Dropout(0.20))

delta.add(Conv2D(256, (3, 3), activation = 'relu'))
delta.add(MaxPooling2D((2, 2)))
delta.add(BatchNormalization())
delta.add(Dropout(0.20))

delta.add(Flatten())
delta.add(Dense(128, activation='relu'))
delta.add(BatchNormalization())
delta.add(Dropout(0.20))

delta.add(Dense(256, activation='relu'))
delta.add(BatchNormalization())
delta.add(Dropout(0.20))

delta.add(Dense(2, activation='softmax')) #2 for cat or dog
delta.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


# *IMAGE HERE*

## Final Model

This may come as a suprise but the best performing model was alpha, despite the lower accuracy score than delta. Here's why:

# IMAGES HERE

While delta performed better what cannot be ignored is that delta labeled more images as 'cat' or 0 and is therefor biased towards that class, while 'dog' or 1 was under represented. With alpha it may have a lower accuracy score but it does not show any signifigant evidence of bias. 

## Next Steps

You may notice that there is no evidence of a test set being used. As this is a kaggle competition there was a test set but the images were not labeled. In the future I will grab 100-200 images from the unlabeled test set and manualy label them cat or dog to get a score out of 100-200 on its test accuracy to produce an accuracy score.